In [17]:
import spires
import numpy as np
import scipy.optimize
import netCDF4

In [18]:
f_sca = 0.5
f_shade = 0.01
dust_concentration = 491
grain_size = 550
x = np.array([f_sca, f_shade, dust_concentration, grain_size])

In [19]:
lut_file = '../tests/data/lut_sentinel2b_b2to12_3um_dust.mat'
interpolator = spires.spires.LutInterpolator(lut_file=lut_file)

In [20]:
interpolator.make_scipy_interpolator()
f = interpolator.interpolator_scipy

In [21]:
spectrum_target = np.array([0.3424, 0.366, 0.3624, 0.38932347, 0.41624767, 0.39567757, 0.07043362, 0.06267947, 0.3792])
spectrum_background = np.array([0.0182, 0.0265, 0.0283, 0.05606749, 0.09543234, 0.12036866, 0.12491679, 0.07888655, 0.1406])
spectrum_shade = np.zeros_like(spectrum_target)
solar_angle = 55.73733298
solar_z = solar_angle

In [22]:
shade = np.zeros(9)

In [23]:
def snow_diff(x):    
    model_reflectances = np.zeros(len(spectrum_target))
    for i in range(0, len(spectrum_target)):            
        pts = np.array([i + 1, solar_z, x[2], x[3]])
        
        model_reflectances[i] = f(pts)[0]

    model_reflectances = model_reflectances * x[0] + shade * x[1] + spectrum_background * (1 - x[0] - x[1])
    
    #return model_reflectances
    return np.linalg.norm(spectrum_target - model_reflectances)

In [24]:
def snow_diff_new(x, spectrum_background, spectrum_target, solar_angle):    
    
    model_reflectances = interpolator.interpolate_all(solar_angle=solar_angle, dust_concentration=x[2], grain_size=x[3])        
    model_reflectances = np.array(model_reflectances)
    model_reflectances = model_reflectances * x[0] + shade * x[1] + spectrum_background * (1 - x[0] - x[1])
    
    #return model_reflectances
    return np.linalg.norm(spectrum_target - model_reflectances)        

In [25]:
#%%timeit
ret = snow_diff(x)
ret

In [26]:
#%%timeit
ret = snow_diff_new(x, spectrum_background, spectrum_target, solar_angle)
ret

In [27]:
solar_angle

In [1]:
%%timeit
ret = spires.core.spectrum_difference(spectrum_background=spectrum_background, 
                                      spectrum_target=spectrum_target,
                                      spectrum_shade=spectrum_shade,
                                      solar_angle=solar_angle,                   
                                      bands=interpolator.bands,
                                      solar_angles=interpolator.solar_angles,
                                      dust_concentrations=interpolator.dust_concentrations,
                                      grain_sizes=interpolator.grain_sizes,        
                                      lut=interpolator.reflectances,
                                      x=x)

In [29]:
x

# Optimize

In [ ]:
constraints = {"type": "ineq", "fun": lambda x: 1 - x[0] + x[1]}

method='SLSQP'
scipy_options = {'disp': False, 'iprint': 100, 'maxiter': 1000, 'ftol': 1e-9}

x0 = np.array([0.5, 0.05, 10, 250])
bounds_fsca = [0, 1]
bounds_fshade = [0, 1]
bounds_dust = [interpolator.dust_concentrations.min(), interpolator.dust_concentrations.max()]
bounds_grain = [interpolator.grain_sizes.min(), interpolator.grain_sizes.max()]
bounds = np.array([bounds_fsca, bounds_fshade, bounds_dust, bounds_grain])

In [ ]:
#%%timeit
res = scipy.optimize.minimize(snow_diff, 
                              x0,                               
                              options=scipy_options, 
                              method=method,
                              bounds=bounds, 
                              constraints=constraints, 
                              )
res

In [ ]:
#%%timeit
res = scipy.optimize.minimize(snow_diff_new, 
                              x0,                               
                              options=scipy_options, 
                              method=method,
                              bounds=bounds, 
                              constraints=constraints, 
                              args=(spectrum_background, spectrum_target, solar_angle))
res.x

In [ ]:
#%%timeit
res = scipy.optimize.minimize(spires.core.spectrum_difference,
                              x0, 
                              method=method, 
                              options=scipy_options, 
                              bounds=bounds, 
                              constraints=constraints, 
                              args=(spectrum_background, 
                                    spectrum_target,
                                    solar_angle,                   
                                    interpolator.bands,
                                    interpolator.solar_angles,
                                    interpolator.dust_concentrations,
                                    interpolator.grain_sizes,        
                                    interpolator.reflectances)
                             )
res

In [ ]:
def index_to_value(value, coords):
    idx = value * coords.size
    l_idx = int(idx)
    r_idx = l_idx + 1
    diff = coords[r_idx] - coords[l_idx]
    dist = idx - l_idx
    return coords[l_idx] + dist * diff

In [ ]:
#%%timeit
method = 'COBYLA'
scipy_options_cobyla = {'disp': False, 'maxiter': 1000, 'rhobeg': 0.1, 'tol': 1e-4}
bounds_dust = [0, 1]
bounds_grain = [0, 1]
bounds_scaled = np.array([bounds_fsca, bounds_fshade, bounds_dust, bounds_grain])
x0_scaled = np.array([0.5, 0.05, 0.01, 0.1])


res = scipy.optimize.minimize(spires.core.spectrum_difference_scaled,
                              x0_scaled, 
                              method=method, 
                              options=scipy_options_cobyla, 
                              bounds=bounds_scaled,
                              constraints=constraints, 
                              args=(spectrum_background, 
                                    spectrum_target,
                                    solar_angle,                   
                                    interpolator.bands,
                                    interpolator.solar_angles,
                                    interpolator.dust_concentrations,
                                    interpolator.grain_sizes,        
                                    interpolator.reflectances)
                             )
res.x[2] = index_to_value(res.x[2], interpolator.dust_concentrations)
res.x[3] = index_to_value(res.x[3], interpolator.grain_sizes)
res

In [ ]:
%%timeit
spires.core.get_fsca(spectrum_background=spectrum_background,
                     spectrum_target=spectrum_target,
                     solar_angle=solar_angle,
                     bands=interpolator.bands,
                     solar_angles=interpolator.solar_angles,
                     dust_concentrations=interpolator.dust_concentrations,
                     grain_sizes=interpolator.grain_sizes,        
                     lut=interpolator.reflectances,
                     max_eval=1000, 
                     x=x0)

In [ ]:
n = 100
results = np.empty((n, 4), dtype=np.double)
spectrum_backgrounds = np.tile(spectrum_background, (n, 1))
spectrum_targets = np.tile(spectrum_target, (n, 1))
solar_angles = np.repeat(solar_angle, n)

In [ ]:
%%timeit
spires.core.get_fscas(spectrum_backgrounds, 
                      spectrum_targets, 
                      solar_angles, 
                      interpolator.bands,
                      interpolator.solar_angles,
                      interpolator.dust_concentrations,
                      interpolator.grain_sizes,        
                      interpolator.reflectances,
                      results, max_eval=100,
                     x=x0)

In [ ]:
results